## py 파일 작업 완료


### KAKAOPAGE SAMPLE URL CRAWLING


In [ ]:
# sample 크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import pickle
import time

driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://page.kakao.com/menu/10011")
# id 추출
URL_ids = [
    "https://page.kakao.com/menu/10011/screen/91",  # 판타지 # 완
    "https://page.kakao.com/menu/10011/screen/64",  # 현판 # 완
    "https://page.kakao.com/menu/10011/screen/70",  # 무협 # 내꺼
    "https://page.kakao.com/menu/10011/screen/100",  # 드라마 # 완
    "https://page.kakao.com/menu/10011/screen/92",  # 로판 # 완완
    "https://page.kakao.com/menu/10011/screen/68",  # 로맨스
    "https://page.kakao.com/menu/10011/screen/77",  # BL
]
genre_list = [
    "crawling_list/kakao/sample/kn_fantasy_sample.pkl",
    "crawling_list/kakao/sample/kn_modern_fantasy_sample.pkl",
    "crawling_list/kakao/sample/kn_muhup_sample.pkl",
    "crawling_list/kakao/sample/kn_drama_sample.pkl",
    "crawling_list/kakao/sample/kn_romance_fantasy_sample.pkl",
    "crawling_list/kakao/sample/kn_romance_sample.pkl",
    "crawling_list/kakao/sample/kn_BL_sample.pkl",
]


# URL 크롤링 함수 정의
def crawling_url(url):
    link_list = []
    driver.get(url)
    time.sleep(5)
    link = "#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col.pt-32pxr > div:nth-child(5) > div > div.flex.w-full.grow.flex-col > div > div > div > div:nth-child({id}) > div > a"

    for i in range(1, 11):
        idx = i + 1
        link_selector = link.format(id=idx)

        link_element = driver.find_element(By.CSS_SELECTOR, link_selector)
        link_meta = link_element.get_attribute("href")
        link_list.append(link_meta)
    return link_list


def save(file_name, list):
    # save pickle
    with open(file_name, "wb") as f:
        pickle.dump(list, f)


for idx, url in enumerate(URL_ids):
    link_list = []
    link_list = crawling_url(url)
    print(f"{url} 크롤링 완료")
    save(genre_list[idx], link_list)
    print(f"{url} 저장완료")

### KAKAOPAGE SAMPLE DETAIL CRAWLING


In [ ]:
# imports
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium import webdriver
import time
import pickle
import re
import json


driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://page.kakao.com/menu/10011")
# 로그인처리
login_icon = r"#__next > div > div.sticky.inset-x-0.left-0.top-0.z-100.flex.w-full.flex-col.items-center.justify-center.bg-bg-a-10 > div > div.flex.h-pc_header_height_px.w-1200pxr.items-center.px-30pxr > div.mr-16pxr.flex.shrink-0.items-center.justify-end.space-x-24pxr > button.pr-16pxr"
driver.find_element(By.CSS_SELECTOR, login_icon).click()
time.sleep(2)
ID_selector = r"#loginId--1"
PWD_selector = r"#password--2"
ID_input = driver.find_element(By.CSS_SELECTOR, ID_selector)
ActionChains(driver).send_keys_to_element(ID_input, "01077399654").perform()
PWD_input = driver.find_element(By.CSS_SELECTOR, PWD_selector)
ActionChains(driver).send_keys_to_element(PWD_input, "rltkghksrnr33!").perform()
login_selector = (
    r"#mainContent > div > div > form > div.confirm_btn > button.btn_g.highlight.submit"
)
driver.find_element(By.CSS_SELECTOR, login_selector).click()
# 카카오 2차 인증 대기
time.sleep(10)
continue_selector = r"#mArticle > div > div.wrap_btn > form > button"
driver.find_element(By.CSS_SELECTOR, continue_selector).click()
Ganre_name = [
    "crawling_list/kakao/sample/kn_fantasy_sample.pkl",
    "crawling_list/kakao/sample/kn_modern_fantasy_sample.pkl",
    "crawling_list/kakao/sample/kn_muhup_sample.pkl",
    "crawling_list/kakao/sample/kn_drama_sample.pkl",
    "crawling_list/kakao/sample/kn_romance_fantasy_sample.pkl",
    "crawling_list/kakao/sample/kn_romance_sample.pkl",
    "crawling_list/kakao/sample/kn_BL_sample.pkl",
]


# 조회수 처리 함수
def view_tran(view):
    view = view.replace(",", "")
    if view[-1] in "억":
        new_view = float(view[:-1])
        new_view *= 100000000
    elif view[-1] == "만":
        new_view = float(view[:-1])
        new_view *= 10000
    else:
        new_view = view
    return int(new_view)


# 크롤링 함수 정의
def crawling_detail(url):
    webnovel = {}
    URL = url + "?tab_type=about"
    driver.get(URL)
    time.sleep(2)
    id_pattern = r"[0-9]+"
    id = re.search(id_pattern, url).group()
    webnovel["id"] = int(id)

    webnovel["type"] = "웹소설"
    webnovel["platform"] = "카카오페이지"

    try:
        title_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > span.font-large3-bold.mb-3pxr.text-ellipsis.break-all.text-el-70.line-clamp-2"
        title_element = driver.find_element(By.CSS_SELECTOR, title_selector).text
    except:
        title_element = "-"
    webnovel["title"] = title_element

    try:
        status_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.mt-6pxr.flex.items-center > span"
        status_element = driver.find_element(By.CSS_SELECTOR, status_selector).text
    except:
        status_element = "-"
    webnovel["status"] = status_element

    try:
        thumbnail_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.overflow-hidden.h-326pxr.w-320pxr.pt-40pxr > div.relative.h-full.min-h-\[inherit\] > div > div > div.jsx-794524088.image-container.relative > img"
        thumbnail_element = driver.find_element(
            By.CSS_SELECTOR, thumbnail_selector
        ).get_attribute("src")
    except:
        thumbnail_element = "-"
    webnovel["thumbnail"] = thumbnail_element

    try:
        genre_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.flex.h-16pxr.items-center.justify-center.all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > div:nth-child(1) > div > span:nth-child(3)"
        genre_element = driver.find_element(By.CSS_SELECTOR, genre_selector).text
    except:
        genre_element = "-"
    webnovel["genre"] = genre_element

    try:
        views_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.flex.h-16pxr.items-center.justify-center.all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > div:nth-child(2) > span"
        views_element = driver.find_element(By.CSS_SELECTOR, views_selector).text
        views_element = view_tran(views_element)
    except:
        views_element = "-"
    webnovel["views"] = views_element

    try:
        rating_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.flex.h-16pxr.items-center.justify-center.all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > div:nth-child(3) > span"
        rating_element = driver.find_element(By.CSS_SELECTOR, rating_selector).text
        rating_element = float(rating_element)
    except:
        rating_element = "-"
    webnovel["rating"] = rating_element

    webnovel["like"] = "-"

    try:
        description_selector = r'//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div[2]/div/div[1]/span'
        description_element = driver.find_element(By.XPATH, description_selector).text
        description_element = description_element.replace("\n", " ")
        description_element = description_element.replace("  ", " ")
    except:
        description_element = "-"
    webnovel["description"] = description_element

    try:
        keywords_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(2) > div.flex.w-full.flex-col.items-center.overflow-hidden > div"

        keywords_element = driver.find_element(By.CSS_SELECTOR, keywords_selector).text
        keywords_element = keywords_element.replace("#", "")
        keywords_element = keywords_element.split("\n")
    except:
        keywords_element = "-"
    webnovel["keywords"] = keywords_element

    try:
        author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(5) > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(1) > span.text-el-70.break-word-anywhere"
        author_element = driver.find_element(By.CSS_SELECTOR, author_selector).text
    except:
        try:
            author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(4) > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(1) > span.text-el-70.break-word-anywhere"
            author_element = driver.find_element(By.CSS_SELECTOR, author_selector).text
        except:
            try:
                author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div.flex.w-full.flex-col.items-center.rounded-12pxr.bg-bg-a-20 > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(1) > span.text-el-70.break-word-anywhere"
                author_element = driver.find_element(
                    By.CSS_SELECTOR, author_selector
                ).text
            except:
                try:
                    author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > span.font-small2.mb-6pxr.text-ellipsis.text-el-70.opacity-70.break-word-anywhere.line-clamp-2"
                    author_element = driver.find_element(
                        By.CSS_SELECTOR, author_selector
                    ).text
                except:
                    author_element = "-"
    webnovel["author"] = author_element

    webnovel["illustrator"] = "-"

    webnovel["original"] = "-"

    try:
        age_rating_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(5) > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(4) > span.text-el-70.break-word-anywhere"
        age_rating_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div.flex.w-full.flex-col.items-center.rounded-12pxr.bg-bg-a-20 > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(4) > span.text-el-70.break-word-anywhere"
        age_rating_element = driver.find_element(
            By.CSS_SELECTOR, age_rating_selector
        ).text
    except:
        age_rating_element = "-"
    webnovel["age_rating"] = age_rating_element

    free_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.overflow-hidden.h-326pxr.w-320pxr.pt-40pxr > div.relative.h-full.min-h-\[inherit\] > div > div > div.flex.flex-col.items-start.space-y-2pxr.absolute.left-4px.top-4px.z-10 > div > span"
    wait_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.flex-1.overflow-hidden.rounded-b-12pxr.bg-bg-a-20.px-24pxr.pt-8pxr > div.mb-8pxr.flex.flex-col.space-y-8pxr > div > div.flex.h-48px.cursor-pointer.items-center.justify-between.px-16pxr > div.flex.flex-wrap.under-320-view\:max-w-\[96px\].under-320-view\:flex-col > span.font-small2-bold.mr-4pxr.text-ellipsis.text-el-70.line-clamp-1 > span"
    try:
        driver.find_element(By.CSS_SELECTOR, free_selector)
        price_element = driver.find_element(By.CSS_SELECTOR, free_selector).text
        price_element = list(price_element)
    except:
        try:
            driver.find_element(By.CSS_SELECTOR, wait_selector)
            price_element = driver.find_element(By.CSS_SELECTOR, wait_selector).text

            price_pattern = r"[0-9][가-힣]+"
            price = re.search(price_pattern, price_element)
            price = price.group() + " 기다리면 무료"
            price_element = [price, "", "100캐시"]
        except:
            price_element = ["유료", "", "100캐시"]
    webnovel["price"] = price_element

    webnovel["url"] = url

    URL = url + "?tab_type=overview"
    driver.get(URL)
    time.sleep(1)
    try:
        episode_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex-1.flex.flex-col > div.rounded-b-12pxr.bg-bg-a-20 > div.flex.h-44pxr.w-full.flex-row.items-center.justify-between.bg-bg-a-20.px-18pxr > div.flex.h-full.flex-1.items-center.space-x-8pxr > span"
        episode_element = driver.find_element(By.CSS_SELECTOR, episode_selector).text
        episode_element = episode_element.strip("전체 ")
        episode_element = episode_element.replace(",", "")
        episode_element = int(episode_element)
    except:
        episode_element = "-"
    webnovel["episode"] = episode_element

    return webnovel


webnovels = []
for ganre in Ganre_name:
    with open(ganre, "rb") as f:
        link_list = pickle.load(f)
    for link in link_list:
        webnovel = crawling_detail(link)
        print(f"{link} 크롤링 완료")
        webnovels.append(webnovel)
    print(f"{ganre} 추가 완료")
with open("../db/kp_wn_sample.json", "w") as f:
    json.dump(webnovels, f, indent=4, ensure_ascii=False)
print("저장 완료")

### KAKAOPAGE TOTAL URL CRAWLING


In [ ]:
# 전체 크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import pickle
import time

driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://page.kakao.com/menu/10011")
# id 추출
URL_ids = [
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=86",
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=120",
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=89",
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=117",
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=87",
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=123",
    "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=125",
]
genre_list = [
    "crawling_list/kakao/all/kakao_web_novels_bl.pkl",
    "crawling_list/kakao/all/kakao_web_novels_d.pkl",
    "crawling_list/kakao/all/kakao_web_novels_f.pkl",
    "crawling_list/kakao/all/kakao_web_novels_mf.pkl",
    "crawling_list/kakao/all/kakao_web_novels_mh.pkl",
    "crawling_list/kakao/all/kakao_web_novels_rf.pkl",
    "crawling_list/kakao/all/kakao_web_novels_r.pkl",
]
# 크롤링 함수 정의
link_list = []


def crawling(cnt):
    min_count = cnt * 24
    max_count = min_count + 24
    print(f"{min_count+1}부터 {max_count}까지 크롤링합니다")

    link = "#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col.pt-32pxr > div.mb-4pxr.flex-col > div > div.flex.grow.flex-col > div > div > div > div:nth-child({id}) > div > a"

    for i in range(min_count, max_count):
        idx = i + 1
        link_selector = link.format(id=idx)

        link_element = driver.find_element(By.CSS_SELECTOR, link_selector)
        link_meta = link_element.get_attribute("href")
        link_list.append(link_meta)


for URL_id, genre in zip(URL_ids, genre_list):
    link_list = []
    driver.get(URL_id)
    time.sleep(5)
    actions = driver.find_element(By.CSS_SELECTOR, "body")
    SCROLL_PAUSE_TIME = 1

    last_height = driver.execute_script("return document.body.scrollHeight")
    cnt = 0
    while True:
        crawling(cnt=cnt)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        cnt += 1
    with open(genre, "wb") as f:
        pickle.dump(link_list, f)

### KAKAOPAGE TOTAL DETAIL CRAWLING


In [1]:
# imports
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium import webdriver
import time
import pickle
import re
import json


driver_path = ChromeDriverManager().install()
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://page.kakao.com/menu/10011")
# 로그인처리
login_icon = r"#__next > div > div.sticky.inset-x-0.left-0.top-0.z-100.flex.w-full.flex-col.items-center.justify-center.bg-bg-a-10 > div > div.flex.h-pc_header_height_px.w-1200pxr.items-center.px-30pxr > div.mr-16pxr.flex.shrink-0.items-center.justify-end.space-x-24pxr > button.pr-16pxr"
driver.find_element(By.CSS_SELECTOR, login_icon).click()
time.sleep(2)
ID_selector = r"#loginId--1"
PWD_selector = r"#password--2"
ID_input = driver.find_element(By.CSS_SELECTOR, ID_selector)
ActionChains(driver).send_keys_to_element(ID_input, "01077399654").perform()
PWD_input = driver.find_element(By.CSS_SELECTOR, PWD_selector)
ActionChains(driver).send_keys_to_element(PWD_input, "rltkghksrnr33!").perform()
login_selector = (
    r"#mainContent > div > div > form > div.confirm_btn > button.btn_g.highlight.submit"
)
driver.find_element(By.CSS_SELECTOR, login_selector).click()
# 카카오 2차 인증 대기
time.sleep(10)
continue_selector = r"#mArticle > div > div.wrap_btn > form > button"
driver.find_element(By.CSS_SELECTOR, continue_selector).click()
Ganre_name = [
    "crawling_list/kakao/all/kakao_web_novels_bl.pkl",
    "crawling_list/kakao/all/kakao_web_novels_d.pkl",
    "crawling_list/kakao/all/kakao_web_novels_f.pkl",
    "crawling_list/kakao/all/kakao_web_novels_mf.pkl",
    "crawling_list/kakao/all/kakao_web_novels_mh.pkl",
    "crawling_list/kakao/all/kakao_web_novels_rf.pkl",
    "crawling_list/kakao/all/kakao_web_novels_r.pkl",
]


# 조회수 처리 함수
def view_tran(view):
    view = view.replace(",", "")
    if view[-1] in "억":
        new_view = float(view[:-1])
        new_view *= 100000000
    elif view[-1] == "만":
        new_view = float(view[:-1])
        new_view *= 10000
    else:
        new_view = view
    return int(new_view)


# 크롤링 함수 정의
def crawling_detail(url):
    webnovel = {}
    URL = url + "?tab_type=about"
    driver.get(URL)
    time.sleep(2)
    id_pattern = r"[0-9]+"
    id = re.search(id_pattern, url).group()
    webnovel["id"] = int(id)

    webnovel["type"] = "웹소설"
    webnovel["platform"] = "카카오페이지"

    try:
        title_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > span.font-large3-bold.mb-3pxr.text-ellipsis.break-all.text-el-70.line-clamp-2"
        title_element = driver.find_element(By.CSS_SELECTOR, title_selector).text
    except:
        title_element = "-"
    webnovel["title"] = title_element

    try:
        status_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.mt-6pxr.flex.items-center > span"
        status_element = driver.find_element(By.CSS_SELECTOR, status_selector).text
    except:
        status_element = "-"
    webnovel["status"] = status_element

    try:
        thumbnail_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.overflow-hidden.h-326pxr.w-320pxr.pt-40pxr > div.relative.h-full.min-h-\[inherit\] > div > div > div.jsx-794524088.image-container.relative > img"
        thumbnail_element = driver.find_element(
            By.CSS_SELECTOR, thumbnail_selector
        ).get_attribute("src")
    except:
        thumbnail_element = "-"
    webnovel["thumbnail"] = thumbnail_element

    try:
        genre_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.flex.h-16pxr.items-center.justify-center.all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > div:nth-child(1) > div > span:nth-child(3)"
        genre_element = driver.find_element(By.CSS_SELECTOR, genre_selector).text
    except:
        genre_element = "-"
    webnovel["genre"] = genre_element

    try:
        views_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.flex.h-16pxr.items-center.justify-center.all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > div:nth-child(2) > span"
        views_element = driver.find_element(By.CSS_SELECTOR, views_selector).text
        views_element = view_tran(views_element)
    except:
        views_element = "-"
    webnovel["views"] = views_element

    try:
        rating_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > div.flex.h-16pxr.items-center.justify-center.all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > div:nth-child(3) > span"
        rating_element = driver.find_element(By.CSS_SELECTOR, rating_selector).text
        rating_element = float(rating_element)
    except:
        rating_element = "-"
    webnovel["rating"] = rating_element

    webnovel["like"] = "-"

    try:
        description_selector = r'//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div[2]/div/div[1]/span'
        description_element = driver.find_element(By.XPATH, description_selector).text
        description_element = description_element.replace("\n", " ")
        description_element = description_element.replace("  ", " ")
    except:
        description_element = "-"
    webnovel["description"] = description_element

    try:
        keywords_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(2) > div.flex.w-full.flex-col.items-center.overflow-hidden > div"

        keywords_element = driver.find_element(By.CSS_SELECTOR, keywords_selector).text
        keywords_element = keywords_element.replace("#", "")
        keywords_element = keywords_element.split("\n")
    except:
        keywords_element = "-"
    webnovel["keywords"] = keywords_element

    try:
        author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(5) > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(1) > span.text-el-70.break-word-anywhere"
        author_element = driver.find_element(By.CSS_SELECTOR, author_selector).text
    except:
        try:
            author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(4) > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(1) > span.text-el-70.break-word-anywhere"
            author_element = driver.find_element(By.CSS_SELECTOR, author_selector).text
        except:
            try:
                author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div.flex.w-full.flex-col.items-center.rounded-12pxr.bg-bg-a-20 > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(1) > span.text-el-70.break-word-anywhere"
                author_element = driver.find_element(
                    By.CSS_SELECTOR, author_selector
                ).text
            except:
                try:
                    author_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.px-18pxr.text-center.bg-bg-a-20.mt-24pxr > a > div > span.font-small2.mb-6pxr.text-ellipsis.text-el-70.opacity-70.break-word-anywhere.line-clamp-2"
                    author_element = driver.find_element(
                        By.CSS_SELECTOR, author_selector
                    ).text
                except:
                    author_element = "-"
    webnovel["author"] = author_element

    webnovel["illustrator"] = "-"

    webnovel["original"] = "-"

    try:
        age_rating_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div:nth-child(5) > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(4) > span.text-el-70.break-word-anywhere"
        age_rating_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex.flex-1.flex-col > div > div > div.flex.w-full.flex-col.items-center.rounded-12pxr.bg-bg-a-20 > div.flex.w-full.flex-col.items-center.overflow-hidden > div > div:nth-child(4) > span.text-el-70.break-word-anywhere"
        age_rating_element = driver.find_element(
            By.CSS_SELECTOR, age_rating_selector
        ).text
    except:
        age_rating_element = "-"
    webnovel["age_rating"] = age_rating_element

    free_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.rounded-t-12pxr.bg-bg-a-20 > div > div.relative.overflow-hidden.h-326pxr.w-320pxr.pt-40pxr > div.relative.h-full.min-h-\[inherit\] > div > div > div.flex.flex-col.items-start.space-y-2pxr.absolute.left-4px.top-4px.z-10 > div > span"
    wait_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.flex.w-320pxr.flex-col > div.flex-1.overflow-hidden.rounded-b-12pxr.bg-bg-a-20.px-24pxr.pt-8pxr > div.mb-8pxr.flex.flex-col.space-y-8pxr > div > div.flex.h-48px.cursor-pointer.items-center.justify-between.px-16pxr > div.flex.flex-wrap.under-320-view\:max-w-\[96px\].under-320-view\:flex-col > span.font-small2-bold.mr-4pxr.text-ellipsis.text-el-70.line-clamp-1 > span"
    try:
        driver.find_element(By.CSS_SELECTOR, free_selector)
        price_element = driver.find_element(By.CSS_SELECTOR, free_selector).text
        price_element = list(price_element)
    except:
        try:
            driver.find_element(By.CSS_SELECTOR, wait_selector)
            price_element = driver.find_element(By.CSS_SELECTOR, wait_selector).text

            price_pattern = r"[0-9][가-힣]+"
            price = re.search(price_pattern, price_element)
            price = price.group() + " 기다리면 무료"
            price_element = [price, "", "100캐시"]
        except:
            price_element = ["유료", "", "100캐시"]
    webnovel["price"] = price_element

    webnovel["url"] = url

    URL = url + "?tab_type=overview"
    driver.get(URL)
    time.sleep(1)
    try:
        episode_selector = r"#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1.flex-col > div > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex-1.flex.flex-col > div.rounded-b-12pxr.bg-bg-a-20 > div.flex.h-44pxr.w-full.flex-row.items-center.justify-between.bg-bg-a-20.px-18pxr > div.flex.h-full.flex-1.items-center.space-x-8pxr > span"
        episode_element = driver.find_element(By.CSS_SELECTOR, episode_selector).text
        episode_element = episode_element.strip("전체 ")
        episode_element = episode_element.replace(",", "")
        episode_element = int(episode_element)
    except:
        episode_element = "-"
    webnovel["episode"] = episode_element

    return webnovel


webnovels = []
for ganre in Ganre_name:
    with open(ganre, "rb") as f:
        link_list = pickle.load(f)
    for link in link_list:
        webnovel = crawling_detail(link)
        print(f"{link} 크롤링 완료")
        webnovels.append(webnovel)
    print(f"{ganre} 추가 완료")
with open("../db/kp_wn_total.json", "w") as f:
    json.dump(webnovels, f, indent=4, ensure_ascii=False)
print("저장 완료")

### 오류 확인


In [11]:
# 오류 확인용
import json

with open("kp_wn_sample.json", "r") as f:
    webnovels = json.load(f)

for novel in webnovels:
    for key, value in novel.items():
        if key == "title" and value == "-":
            print(novel)
        if key == "discription" and value == "-":
            print(novel)
        if key == "status" and value == "-":
            print(novel)
        if key == "price" and value == "-":
            print(novel)
        if key == "age_rating" and value == "-":
            print(novel)
        if key == "thumbnail" and value == "-":
            print(novel)
        if key == "ganre" and value == "-":
            print(novel)
        if key == "view" and value == "-":
            print(novel)
        if key == "author" and value == "-":
            print(novel)
        if key == "episode" and value == "-":
            print(novel)